# Álgebra Matricial con Python

Autor: Pedro González Rodelas

Fecha de la primera versión: 22,25/08/2017

Fecha de la última revisión: 8,10,17/05/2022


En esta práctica se introducen los conceptos básicos relacionados con las matrices de números reales, como son la traspuesta, inversa, etc. de una dada; los tipos fundamentales de matrices: simétricas, inversibles, ortogonales, etc.; así como las técnicas elementales del algebra matricial, como son la suma de matrices y el producto por un escalar, el producto de matrices, etc.

Para completar esta práctica sobre Álgebra Lineal Numérica se puede consultar alguna de las prácticas desarrolladas por Raúl López Briega en su Blog relacionadas con este tema:
* [Álgebra Lineal con Python](https://relopezbriega.github.io/blog/2015/06/14/algebra-lineal-con-python/)
* [Más Álgebra Lineal con Python](https://relopezbriega.github.io/blog/2016/02/10/mas-algebra-lineal-con-python/)

## Carga de los módulos y Definición de ciertas funciones empleadas

Antes de empezar a realizar cualquier cálculo numérico o simbólico debemos de cargar los correspondientes módulos de Python que implementan la mayoría de funciones y procedimientos necesarios para ello: NumPy y SymPy, que serán cargados con los pseudónimos np y sp, respectivamente. Además cargaremos la función del módulo random que nos permitirá generar números aleatorios, cada vez que los necesitemos.

In [1]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp

In [2]:
from random import random 
# Importamos la función generadora de números pseudoaleatorios

También definiremos cierta función propia, `mychop`, que nos permitirá cambiar por $0$ cualquier valor que obtengamos, después de ciertos cálculos que involucren errores de redondeo, propios del cálculo con números en coma flotante. A su vez usaremos una versión vectorizada de la misma, que podremos aplicar a cualquier lista, tupla o array con valores numéricos en coma flotante.

In [3]:
def mychop(expr, max=10**(-15)): 
    if abs(expr) > max:
      return expr 
    else:
      return 0

In [4]:
chop_vec = np.vectorize(mychop)

## Introducción

Hay varias maneras de introducir y trabajar con *objetos* pertenecientes a *clases* concretas que nos permitirán trabajar con ellos de manera parecida a  como si fueran las *matrices* que conocemos del Álgebra Lineal. Pero habrá que tener mucho cuidado, porque no todas esas *clases de objetos* tendrán las mismas *propiedades*, ni se podrán realizar todas las *operaciones* que conocemos, ni tampoco el *resultado* final será el esperado en algunos casos; por lo tanto, más vale realizar un repaso exhaustivo de todas estas posibilidades antes de aventurarnos a trabajar con las supuestas matrices en Python.

In [5]:
u = [1,2,3,4]; v = [5,6,7,8]   # aquí tenemos simplemente dos listas 
# de números con el mismo número de elementos cada una 

In [6]:
u[0] # podemos acceder a cada uno de los elementos de estas listas
# con su correspondiente índice (en Python el primero será siempre 0)

1

In [7]:
u[2]*v[2] # por supuesto que también podemos operar con éstos

21

In [8]:
u + v # pero fíjese qué ocurre cuando se intentan sumar

[1, 2, 3, 4, 5, 6, 7, 8]

In [9]:
# u*v # o multiplicar

Vemos pues que cualquier operación elemento a elemento, tal y cómo estamos acostumbrados a considerar, no estaría definida por defecto en Python para las listas de elementos, a no ser que las programemos nosotros expresamente, mediante la correspondiente lista por comprensión.

In [10]:
[u[i]+v[i] for i in range(len(v))] #  suma elemento a elemento

[6, 8, 10, 12]

In [11]:
[u[i]*v[i] for i in range(len(v))] #  producto elemento a elemento

[5, 12, 21, 32]

In [12]:
sum([u[i]*v[i] for i in range(len(v))]) # producto escalar de u y v

70

In [13]:
A = [u,v]  # también podemos definir una lista formada por otras 
# nuevas sublistas, cada una de ellas con sus propios elementos.

In [14]:
A[0] # podemos acceder pues a cada una de estas sublistas completas

[1, 2, 3, 4]

In [15]:
A[1][0] # también podremos acceder fácilmente a cada uno de
# los elementos de estas sublistas, usando un doble índice

5

In [16]:
2*A # pero vemos que ocurre algo raro cuando intentamos operar con
# los elementos de esta estructura; por ejemplo al intentar 
# multiplicar por un valor entero lo que obtenemos es otra lista, 
# con copias de las anteriores.

[[1, 2, 3, 4], [5, 6, 7, 8], [1, 2, 3, 4], [5, 6, 7, 8]]

In [17]:
# A + 2  # incluso nos da error si intentamos sumarle cualquier número

In [18]:
A + [2]  # sin embargo lo admite cuando incluimos ese número entre []
# pero lo que realiza es una simple concatenación

[[1, 2, 3, 4], [5, 6, 7, 8], 2]

In [19]:
 A + [[9,10,11,12]] # de esta manera hemos conseguido incluso añadir
# otra sublista, con el mismo número de elementos que las anteriores

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]

In [20]:
A + [[9,10,11]] # aunque este último no es un requisito indispensable

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11]]

Llegamos pues a la conclusión de que las listas de sublistas, al contrario de lo que suele ocurrir con otros sistemas CAS como Mathematica o MATLAB por ejemplo, no es lo más adecuado en este caso para trabajar con matrices en Python y que tendremos que usar otras *clases* de *objetos* que sí nos sirvan.

No obstante, como veremos a lo largo de esta práctica, existen muchas opciones y posibilidades para trabajar eficientemente con matrices y vectores, haciendo uso de las estructuras y *clases de objetos* adecuados, incluidos en alguno de los paquetes por excelencia para el cálculo científico en Python: SymPy (cuando necesitemos trabajar de manera simbólica) y NumPy (cuando nuestros cálculos sean fundamentalmente numéricos).

## Álgebra matricial simbólica

In [21]:
from sympy import Matrix, symbols, solve, simplify 

In [22]:
m11, m12, m21, m22 = symbols('m11, m12, m21, m22')
b1, b2 = symbols('b1, b2')

In [23]:
# También se podrían haber generado de golpe las variables simbólicas
# de la matriz de orden 2 con la siguiente orden
symbols('m(1:3)(1:3)')

(m11, m12, m21, m22)

In [24]:
A = Matrix([[m11,m12],[m21,m22]])

In [25]:
b = Matrix([[b1],[b2]])

In [26]:
A

Matrix([
[m11, m12],
[m21, m22]])

In [27]:
b

Matrix([
[b1],
[b2]])

In [28]:
type(A),type(b)  # comprobemos el tipo de estos nuevos objetos

(sympy.matrices.dense.MutableDenseMatrix,
 sympy.matrices.dense.MutableDenseMatrix)

In [29]:
A*b # Vemos que ahora con las instancias de este tipo de objetos
# sí que podemos realizar las operaciones matriciales habituales
# y además de manera simbólica en este caso.

Matrix([
[b1*m11 + b2*m12],
[b1*m21 + b2*m22]])

In [30]:
A**2 # aquí hemos obtenido el cuadrado de la matriz A

Matrix([
[ m11**2 + m12*m21, m11*m12 + m12*m22],
[m11*m21 + m21*m22,  m12*m21 + m22**2]])

In [31]:
A.det() # este sería el determinante de la matriz cuadrada 2x2

m11*m22 - m12*m21

In [32]:
Ainv = A.inv() # incluso la inversa de ciertas matrices también podemos 
# obtenerlo de manera simbólica (haría falta que no sea singular).

In [33]:
simplify(Ainv*b)   # He aquí pues la solución del correspondiente 
# sistema lineal Ax = b, despejando x = A^(-1)b

Matrix([
[ (b1*m22 - b2*m12)/(m11*m22 - m12*m21)],
[(-b1*m21 + b2*m11)/(m11*m22 - m12*m21)]])

Hagamos ahora el intento de resolver simbólicamente cualquier sistema lineal compatible determinado de orden dos mediante la orden de resolución simbólica de ecuaciones `solve`; para ello usaremos la matriz y el vector de términos independientes ya definidos previamente y definamos también un vector de incógnitas también simbólico para poder construir el sistema final de dos ecuaciones (igualado a cero), indicando así mismo las dos incógnitas del mismo, dentro de la orden de resolución simbólica de ecuaciones `solve` (que ya hemos precargado previamente dentro del módulo SymPy).

In [34]:
?solve   # de esta manera podemos obtener información sobre la orden

Object `solve   # de esta manera podemos obtener información sobre la orden` not found.


In [35]:
x1, x2 = symbols('x1, x2')

In [36]:
# Si definimos ahora una matriz simbólica
x = Matrix([[x1], [x2]])  # de esta forma
# nótese que se convierte en un vector columna

In [37]:
A*x

Matrix([
[m11*x1 + m12*x2],
[m21*x1 + m22*x2]])

In [38]:
A*x-b

Matrix([
[-b1 + m11*x1 + m12*x2],
[-b2 + m21*x1 + m22*x2]])

In [39]:
# Así es como podremos resolver el correspondiente sistema lineal,
solve(A*x-b,[x1,x2])   #  con incógnitas x1, x2

{x1: (b1*m22 - b2*m12)/(m11*m22 - m12*m21),
 x2: (-b1*m21 + b2*m11)/(m11*m22 - m12*m21)}

Vemos claramente la solución genérica (simbólica) de un sistema 2x2 que para que sea compatible y determinado bastará con que su determinante (que justo es el valor que aparece en el denominador de las soluciones) sea distinto de cero.

In [40]:
A.LUsolve(b)   # Esta es otra posibilidad usando LUsolve

Matrix([
[(b1 - m12*(-b1*m21/m11 + b2)/(m22 - m12*m21/m11))/m11],
[               (-b1*m21/m11 + b2)/(m22 - m12*m21/m11)]])

In [41]:
simplify(_)

Matrix([
[ (b1*m22 - b2*m12)/(m11*m22 - m12*m21)],
[(-b1*m21 + b2*m11)/(m11*m22 - m12*m21)]])

**Ejercicio:** Intente repetir el mismo proceso para obtener de manera simbólica las soluciones genéricas de un sistema compatible y determinado 3x3.

**Ayuda:** Con el objeto de mecanizar el proceso de creación de la matriz simbólica, se puede usar la siguiente orden que permite generar de golpe todos los símbolos necesarios

    M = Matrix(3, 3, symbols('m(1:4)(1:4)'))

In [42]:
M = Matrix(3, 3, symbols('m(1:4)(1:4)'))
M

Matrix([
[m11, m12, m13],
[m21, m22, m23],
[m31, m32, m33]])

También es posible generar de manera sistemática matrices de cualquier orden, siempre que tengamos la ley o función generadora, dependiendo de la fila y columna que ocupe cada elemento de la misma.

Veámos como ejemplo la generación de una matriz de Toeplitz, con diagonales constantes, por ejemplo a partir de un vector de $2n-1$ datos $a_k, \; k = 1,\ldots, 2n-1$, a partir de la siguiente fórmula generadora:

$$T_{i,j} = a_{i-j+(n-1)},\; i,j = 0,\ldots, n-1 $$

In [43]:
def toeplitz(n):   # definimos una función dependiendo de 'n'
    a = symbols('a:'+str(2*n)) # generamos los símbolos necesarios
    f = lambda i,j: a[i-j+n-1] # función generadora (anónima)
    return Matrix(n,n,f)       # devuelve la matriz final

In [44]:
M = toeplitz(5) # aquí hacemos una llamada concreta a la anterior función

In [45]:
M

Matrix([
[a4, a3, a2, a1, a0],
[a5, a4, a3, a2, a1],
[a6, a5, a4, a3, a2],
[a7, a6, a5, a4, a3],
[a8, a7, a6, a5, a4]])

In [46]:
a2, a7 = symbols('a2, a7')  # los usaremos de nuevo como símbolos
M.subs(a2,0).subs(a7,1) # ahora podemos sustituirlos por algún valor 
# numérico por ejemplo.

Matrix([
[a4, a3,  0, a1, a0],
[a5, a4, a3,  0, a1],
[a6, a5, a4, a3,  0],
[ 1, a6, a5, a4, a3],
[a8,  1, a6, a5, a4]])

In [47]:
M.subs({a2:0, a7:1})  # esta sería otra manera de hacerlo

Matrix([
[a4, a3,  0, a1, a0],
[a5, a4, a3,  0, a1],
[a6, a5, a4, a3,  0],
[ 1, a6, a5, a4, a3],
[a8,  1, a6, a5, a4]])

## Álgebra matricial numérica

No cabe la menor duda pues de que las operaciones vectoriales y/o matriciales son indispensables en la computación científica. Por ello algunos módulos de Python utilizan diferentes estructuras y clases de objetos que implementan todas las operaciones habituales relacionadas. Entre estos módulos por excelencia que se basan en este tipo de estructuras vectoriales (también denominados arrays) cabe destacar sin duda `NumPy`.

A diferencia de las variables genéricas en Python, que son de tipado dinámico y pueden llegar a almacenar sucesivamente todo tipo de datos uno tras otro, las variables y arrays definidos en NumPy van a ser de tipo estático y homogéneo (determinado desde su inicio o definición) y las operaciones posibles entre ellas estarán también predeterminadas de antemano, según el tipo de los operandos. Esto, aunque parezca que le quita cierta flexibilidad a las posibilidades de Python, hace que muchas operaciones de cómputo intensivo, tan necesarias en cálculo científico, se puedan realizar con una rapidez y eficiencia casi comparable al de los lenguajes compilados (como Fortran, C, C++ y otros), aún manteniendo la característica de lenguaje interpretado que tantas ventajas tiene de cara a interactividad y facilidad de programación.

In [48]:
import numpy as np  # para ello lo primero que debemos hacer (si no) 
# lo hubiéramos hecho ya antes, es importar dicho módulo

In [49]:
# help(np.ndarray)  # para obtener la máxima información sobre la clase
# de objetos ndarray que vamos a emplear.

In [50]:
datos = np.array([[1,2],[3,4],[5,6]])  # he aquí un primer ejemplo

In [51]:
type(datos)   # comprobemos ahora el tipo de dato de que se trata

numpy.ndarray

In [52]:
datos

array([[1, 2],
       [3, 4],
       [5, 6]])

Ahora, para extraer alguno de los elementos de estos *arrays* debemos indicar los índices de la correspondiente fila y columna mediante corchetes; mientras que si sólo indicamos un índice, éste corresponderá a toda una fila. Veámoslo:

In [53]:
datos[0], datos[0,0]

(array([1, 2]), 1)

In [54]:
datos[1]

array([3, 4])

In [55]:
datos.ndim   # para obtener el número de dimensiones del array

2

In [56]:
datos.shape, datos.size # para obtener sus 'forma' y 'tamaño'

((3, 2), 6)

Vemos que la salida de la sentencia 'shape' es una tupla, donde el primer elemento es el número de filas y el segundo el de columnas y así sucesivamente. Así pues, también podremos extraer cada uno de estos valores empleando corchetes, como sigue:

In [57]:
datos.shape[0],datos.shape[1]

(3, 2)

In [58]:
datos.dtype, datos.nbytes # estas órdenes nos proporcionan tanto el 
# tipo de datos que contiene el array, así como el total de bytes

(dtype('int32'), 24)

In [59]:
datos[0,0] = 10  # así cambiaríamos el valor de un elemento concreto

In [60]:
datos[1] = [30,40] # y así una fila completa

In [61]:
datos # aquí vemos como ha quedado el array después de los cambios

array([[10,  2],
       [30, 40],
       [ 5,  6]])

In [62]:
datos[2,0] = 15.4 # veámos qué pasa ahora al intentar darle un valor
# con decimales a una de las posiciones del array

In [63]:
datos  # vemos que se ha convertido automáticamente a tipo entero
# al ser el tipo de datos fijo del array desde su creación

array([[10,  2],
       [30, 40],
       [15,  6]])

In [64]:
datosbis = np.array([[1.,2],[3,4],[5,6]]) # véamos qué pasa ahora al 
# haber introducido un valor con el punto decimal en una posición

In [65]:
datosbis # resulta que todos los datos se han reconvertido, ya que
# todos los datos de cualquier array deben ser del mismo tipo

array([[1., 2.],
       [3., 4.],
       [5., 6.]])

In [66]:
datosbis.dtype, datosbis.nbytes # vemos que aunque ahora el tipo de 
# dato ha cambiado a float, se emplean el mismo número de bytes 

(dtype('float64'), 48)

In [67]:
datosbis.shape, datosbis.size  # coincide con el ejemplo anterior

((3, 2), 6)

No obstante, como veremos, también se puede especificar el tipo concreto de dato que queremos que se considere cuando se va a definir un array de NumPy. Veánse los ejemplos siguientes:

In [68]:
np.array([1,1,1], dtype = np.int16)

array([1, 1, 1], dtype=int16)

In [69]:
_.nbytes

6

In [70]:
np.array([1,1,1], dtype = np.float16)

array([1., 1., 1.], dtype=float16)

In [71]:
_.nbytes

6

In [72]:
np.array([1,1,1], dtype = np.float64)

array([1., 1., 1.])

In [73]:
_.nbytes

24

In [74]:
np.array([1,1,1], dtype = complex)

array([1.+0.j, 1.+0.j, 1.+0.j])

In [75]:
_.nbytes

48

In [76]:
datosbis.astype(int)  # aunque también es posible cambiar el tipo
# de datos de todo el array completo mediante este procedimiento

array([[1, 2],
       [3, 4],
       [5, 6]])

In [77]:
datos.astype(complex)

array([[10.+0.j,  2.+0.j],
       [30.+0.j, 40.+0.j],
       [15.+0.j,  6.+0.j]])

In [78]:
datos + datosbis # Nótese que ahora sí que podemos operar con arrays
# que posean la misma estructura (número de filas y columnas)

array([[11.,  4.],
       [33., 44.],
       [20., 12.]])

In [79]:
v1 = np.array([1,2,3], dtype = float)
v2 = np.array([4,5,6]) # podemos generar arrays de distintos tipos

In [80]:
5*v1  # y ya sí que podemos multiplicar todos sus elementos por un escalar

array([ 5., 10., 15.])

In [81]:
v1 + v2 # y operar con ellos como si fueran vectores o matrices
# siempre y cuando tengan la misma estructura (filas y columnas)

array([5., 7., 9.])

In [82]:
_.dtype  # El resultado se ha convertido en un array de tipo float64

dtype('float64')

In [83]:
v2.dtype # aunque v2 fuera un array de números enteros

dtype('int32')

In [84]:
np.sqrt(v2) # ahora podemos aplicar también a todos los elementos 
# del array al mismo tiempo cualquier función de NumPy 

array([2.        , 2.23606798, 2.44948974])

In [85]:
v2**2  # elevar todos sus componentes al cuadrado, u otra potencia

array([16, 25, 36], dtype=int32)

In [86]:
np.log(v2)  # o aplicarles cualquier función de NumPy, ya vectorizadas

array([1.38629436, 1.60943791, 1.79175947])

In [87]:
np.sin(v1)  # para poder actuar en todas las componentes a la vez  

array([0.84147098, 0.90929743, 0.14112001])

### Generando y modificando arrays y matrices especiales

In [88]:
np.identity(3) # para generar la matriz identidad de orden n

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [89]:
np.eye(3)   # esta sería otra manera de obtenerla

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [90]:
np.zeros([2,3])

array([[0., 0., 0.],
       [0., 0., 0.]])

In [91]:
np.ones([2,3])  # generación de un array 2x3 formado sólo con 1's

array([[1., 1., 1.],
       [1., 1., 1.]])

In [92]:
np.ones(6).reshape(2,3) # una forma equivalente de obtenerlo

array([[1., 1., 1.],
       [1., 1., 1.]])

In [93]:
np.diag(range(1,10,2))  # así generamos un array con los elementos
# indicados por la orden de Python range

array([[1, 0, 0, 0, 0],
       [0, 3, 0, 0, 0],
       [0, 0, 5, 0, 0],
       [0, 0, 0, 7, 0],
       [0, 0, 0, 0, 9]])

In [94]:
np.arange(1,10,2)  # esta orden de NumPy es una variante de range
# que genera directamente un array

array([1, 3, 5, 7, 9])

In [95]:
np.array(range(1,10,2)) # esta sería una manera equivalente

array([1, 3, 5, 7, 9])

Generemos ahora por ejemplo una matriz con elementos racionales que contienen los valores $\frac{1}{i+j}$, donde el índice $i$ varía entre 1 y 5 (de uno en uno), mientras que el índice $j$ variará de 0 a 4 (de dos en dos).

**Nota**: Con otros lenguajes como Mathematica generaríamos esta matriz con la siguiente orden: 
        
        Table[1/(i+j),{i,5},{j,0,4,2}]//MatrixForm
        
pero hay que recordar que en Python los rangos de índices comienzan por defecto en 0, en vez de en 1, y además el último valor indicado no se llega a alcanzar, independientemente del paso que se indique; así que el código eqivalente en Python sería el que aparece a continuación:

In [96]:
mat = np.array([[sp.Rational(1,i+j) 
                 for i in range(1,6)] for j in range(0,6,2)])
mat

array([[1, 1/2, 1/3, 1/4, 1/5],
       [1/3, 1/4, 1/5, 1/6, 1/7],
       [1/5, 1/6, 1/7, 1/8, 1/9]], dtype=object)

In [97]:
2*mat  # ya podemos multiplicar todos los elementos del array por un número

array([[2, 1, 2/3, 1/2, 2/5],
       [2/3, 1/2, 2/5, 1/3, 2/7],
       [2/5, 1/3, 2/7, 1/4, 2/9]], dtype=object)

In [98]:
 # o incluso por cualquier valor
a = symbols('a')  # que tengamos definido simbólicamente
a*mat

array([[a, a/2, a/3, a/4, a/5],
       [a/3, a/4, a/5, a/6, a/7],
       [a/5, a/6, a/7, a/8, a/9]], dtype=object)

In [99]:
?np.ndarray

In [100]:
np.linspace(0,10,11) # así generamos un array de puntos igualmente espaciados 
# (partición uniforme) con los extremos y número de nodos indicados

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

## Operaciones vectoriales y matriciales

Todas las matrices para las que el número de filas coincide con el de columnas se denominan matrices cuadradas. Veámos a continuación algunos ejemplos y cómo podemos operar con ellas.

In [101]:
matriz1 = np.random.randn(3,3) # así generamos matrices 3x3 con 
matriz2 = np.random.randn(3,3) # elementos aleatorios obtenidos a 
# partir de una distribución normal de media 0 y varianza 1

In [102]:
matriz0 = np.zeros([3,3])    # matriz nula
matrizI = np.identity(3)     # matriz identidad

Las llamadas matrices *escalares* se pueden construir a partir de la matriz identidad de orden n, que se obtiene con la orden  `np.identity(n)` que define una matriz de orden n  con unos en la diagonal y ceros en el resto de posiciones.

In [103]:
a

a

In [104]:
a*matrizI

array([[1.0*a, 0, 0],
       [0, 1.0*a, 0],
       [0, 0, 1.0*a]], dtype=object)

Como hemos visto en el último ejemplo, el producto de una matriz por un escalar se realiza de una forma inmediata, sin más que separar ambos por un `*` (como para el producto entre dos números cualesquiera). 

In [105]:
matriz1, matriz2

(array([[-0.98177445,  0.96752798,  0.29957593],
        [ 0.93618402, -0.34696359,  1.10039057],
        [-0.55624758,  0.40423114, -0.35143563]]),
 array([[ 0.02097048,  1.83439534, -0.14799319],
        [-0.28492758,  1.58549718, -1.44415278],
        [ 0.99451182, -0.53189256,  0.19888235]]))

En cuanto a la suma y resta de vectores (arrays unidimensionales) matrices (arrays bidimensionales) o bien arrays multidimensionales en general, es necesario de antemano que ambos tengan los mismos órdenes/dimensiones para poder realizar dicha operación; es decir, el mismo número de filas, de columnas, etc., ya que estas operaciones entre vectores, matrices o arrays en general se realizan componente a componente. No obstante, en el caso de matrices cuadradas y del mismo orden, no habrá ningún problema tanto para la suma, ni para el producto. Veamos algunos ejemplos:

Nótese que `mat0` es el elemento neutro para  la suma de matrices. En cuanto a la matriz identidad, lo será para la multiplicación.

In [106]:
matriz0 + matriz1  # la matriz0 es el elemento neutro para la suma

array([[-0.98177445,  0.96752798,  0.29957593],
       [ 0.93618402, -0.34696359,  1.10039057],
       [-0.55624758,  0.40423114, -0.35143563]])

In [107]:
matriz1 + matriz2

array([[-0.96080397,  2.80192331,  0.15158274],
       [ 0.65125644,  1.23853359, -0.34376221],
       [ 0.43826424, -0.12766142, -0.15255328]])

La multiplicación matricial como tal (no elemento a elemento que proporciona `*`) se obtendrá a partir de la orden `np.dot()` o bien con el operador `@`

In [108]:
np.dot(matrizI,matriz1) # y matrizI el elemento neutro para el producto 

array([[-0.98177445,  0.96752798,  0.29957593],
       [ 0.93618402, -0.34696359,  1.10039057],
       [-0.55624758,  0.40423114, -0.35143563]])

In [109]:
np.dot(matriz1,matriz2) # así haríamos el producto matricial

array([[ 0.00166812, -0.4262918 , -1.19238191],
       [ 1.21284315,  0.58193224,  0.58136784],
       [-0.47634827, -0.19254463, -0.57134502]])

In [110]:
matriz1.dot(matriz2)  # esta sería otra forma equivalente de pedirlo

array([[ 0.00166812, -0.4262918 , -1.19238191],
       [ 1.21284315,  0.58193224,  0.58136784],
       [-0.47634827, -0.19254463, -0.57134502]])

In [111]:
matriz1 @ matriz2     # y aquí usando el operador @

array([[ 0.00166812, -0.4262918 , -1.19238191],
       [ 1.21284315,  0.58193224,  0.58136784],
       [-0.47634827, -0.19254463, -0.57134502]])

In [112]:
np.linalg.matrix_power(matriz1,2) # y así obtendríamos la potencia
# entera de una matriz cuadrada

array([[ 1.70302691, -1.16449331,  0.66526101],
       [-1.85603292,  1.4709801 , -0.48805373],
       [ 1.12002962, -0.82049981,  0.40168076]])

In [113]:
np.linalg.matrix_power(matriz1,-2) # aunque en algunas ocasiones
# también podremos calcular ciertas raíces de estas matrices

array([[ 19.56410765,  -8.02346782, -42.1506909 ],
       [ 20.43554649,  -6.27136897, -41.46511142],
       [-12.80872408,   9.56198296,  35.32134893]])

In [114]:
np.linalg.matrix_power(matriz1,0) # vemos cómo al elevar a 0 una
# matriz cuadrada obtenemos la matriz identidad correspondiente

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [115]:
np.linalg.matrix_power(matriz1,-1) # y esta potencia -1 deberá ser
# equivalente a la correspondiente matriz inversa de dicha matriz

array([[-3.27276359,  4.67405075, 11.84524217],
       [-2.86938497,  5.18641028, 13.79336027],
       [ 1.87963883, -1.43246962, -5.72845017]])

In [116]:
# aquí lo comprobamos sin problema
np.dot(np.linalg.matrix_power(matriz1,-1),matriz1) # multiplicando
# pues estas dos matrices obtenemos, salvo los inevitables errores 
# de redondeo, una matriz que se puede considerar la matriz identidad

array([[ 1.00000000e+00,  4.47265712e-16,  2.35496097e-16],
       [ 1.50054526e-15,  1.00000000e+00,  1.28265152e-15],
       [-5.26430126e-16,  5.63855963e-17,  1.00000000e+00]])

In [117]:
# para este tipo de casos habíamos introducido la función 'chop_vec'
chop_vec(_)  # que era la versión vectorizada de 'mychop'

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.50054526e-15, 1.00000000e+00, 1.28265152e-15],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

Pero no toda matriz cuadrada podrá tener un elemento inverso para la multiplicación; sólo aquellas que lo posean se denominarán inversibles. Python  dispone de una orden directa para su cálculo en caso de que ésta exista. Esta sentencia es `np.linalg.inv(matriz)`.

In [118]:
matriz1inv = np.linalg.inv(matriz1) # equivaldría a la inversa de la 
# matriz (siempre y cuando ésta sea regular)
matriz1inv

array([[-3.27276359,  4.67405075, 11.84524217],
       [-2.86938497,  5.18641028, 13.79336027],
       [ 1.87963883, -1.43246962, -5.72845017]])

Se comprueba fácilmente que este conjunto de las matrices cuadradas de orden n, junto con las operaciones de suma y producto (cada una con su correspondiente elemento neutro) tiene estructura de anillo no conmutativo (satisfaciendo pues los correspondientes axiomas y propiedades) con divisores de cero (justo esto es lo que impide que no toda matriz cuadrada tenga inversa) como veremos a continuación:

In [119]:
mat1 = np.array([[1,0],[4,0]])  # aquí tenemos dos matrices
mat2 = np.array([[0,0],[4,5]])  # cuadradas de orden 2

In [120]:
np.dot(mat1,mat2)  # cuyo producto matricial es la matriz nula

array([[0, 0],
       [0, 0]])

In [121]:
np.dot(mat2,mat1)  # pero vemos que el producto no es conmutativo

array([[ 0,  0],
       [24,  0]])

Trabajemos ahora con algunos ejemplos de matrices (arrays bidimensionales) rectangulares (no cuadradas), donde no coincida el número de filas y columnas.

In [122]:
matriz1bis = np.random.randn(2,3) # así generamos matrices 2x3 con 
matriz2bis = np.random.randn(2,3) # elementos aleatorios obtenidos a 
# partir de una distribución normal de media 0 y varianza 1
matriz1bis

array([[-0.61083798, -0.84099153, -0.12511128],
       [-0.80759465, -0.60118669,  0.5557704 ]])

In [123]:
matriz2bis

array([[-0.06439003, -0.21816218, -1.77219037],
       [-0.04526294,  1.09953893, -0.24614132]])

In [124]:
np.linalg.matrix_rank(matriz2bis) # para saber el rango de una matriz
# osea número de filas o columnas linealmente independientes

2

In [125]:
matriz1bisT = matriz1bis.T  # así obtenemos la matriz traspuesta
matriz1bisT

array([[-0.61083798, -0.80759465],
       [-0.84099153, -0.60118669],
       [-0.12511128,  0.5557704 ]])

De manera que la matriz resultante tendrá un número de filas y columnas coincidentes con el número de columnas y filas de la primera.

In [126]:
matriz1bis.shape,matriz1bisT.shape # vemos cómo se intercambian el 
# número de filas y columnas al obtener la matriz traspuesta

((2, 3), (3, 2))

In [127]:
np.transpose(matriz1bis)  # otra forma de trasponer una matriz

array([[-0.61083798, -0.80759465],
       [-0.84099153, -0.60118669],
       [-0.12511128,  0.5557704 ]])

In [128]:
np.rot90(matriz1bis) # aquí se hace algo equivalente a 'rotar' 
# los elementos de las dos primeras dimensiones de la matriz o array 

array([[-0.12511128,  0.5557704 ],
       [-0.84099153, -0.60118669],
       [-0.61083798, -0.80759465]])

In [129]:
np.sort(matriz1bis) # ordena los elementos de cada una de las filas
# equivalente a la siguiente orden 'np.sort(matriz1,1)'

array([[-0.84099153, -0.61083798, -0.12511128],
       [-0.80759465, -0.60118669,  0.5557704 ]])

In [130]:
np.sort(matriz1bis,0) # compare con la ordenación anterior

array([[-0.80759465, -0.84099153, -0.12511128],
       [-0.61083798, -0.60118669,  0.5557704 ]])

In [131]:
matriz1bis

array([[-0.61083798, -0.84099153, -0.12511128],
       [-0.80759465, -0.60118669,  0.5557704 ]])

In [132]:
np.ndarray.sort(matriz1bis) # con esta orden además de realizar dicha
# ordenación por filas, finalmente se modifica la propia matriz 

In [133]:
matriz1bis # compárese con la matriz original más arriba

array([[-0.84099153, -0.61083798, -0.12511128],
       [-0.80759465, -0.60118669,  0.5557704 ]])

In [134]:
?np.fliplr

In [135]:
?np.flipud

In [136]:
np.fliplr(matriz1bis)

array([[-0.12511128, -0.61083798, -0.84099153],
       [ 0.5557704 , -0.60118669, -0.80759465]])

In [137]:
np.flipud(matriz1bis)

array([[-0.80759465, -0.60118669,  0.5557704 ],
       [-0.84099153, -0.61083798, -0.12511128]])

In [138]:
v1,v2

(array([1., 2., 3.]), array([4, 5, 6]))

In [139]:
np.dot(v1,v2)   # este sería el producto escalar de vectores

32.0

In [140]:
np.inner(v1,v2) # y esta sería una manera equivalente de hacerlo

32.0

In [141]:
np.dot(matriz1bis,matriz1bisT)  # también sirve para multiplicar 
# matrices con las dimensiones adecuadas

array([[1.09604261, 0.97687477],
       [0.97687477, 1.3225153 ]])

In [142]:
np.dot(matriz1bis,v1)  # y por supuesto matrices y vectores

array([-2.43800133, -0.34265685])

In [143]:
np.cross(v1,v2)  # este sería el producto vectorial de vectores

array([-3.,  6., -3.])

In [144]:
np.outer(v1,v2)  # producto exterior de vectores

array([[ 4.,  5.,  6.],
       [ 8., 10., 12.],
       [12., 15., 18.]])

In [145]:
np.tensordot(matriz1bis,matriz1bisT,0) # producto tensorial 
# de matrices según la dimensión indicada

array([[[[ 0.70726675,  0.67918026],
         [ 0.51370956,  0.50559292],
         [ 0.10521753, -0.46739819]],

        [[ 0.51370956,  0.49330948],
         [ 0.37312303,  0.36722766],
         [ 0.07642272, -0.33948566]],

        [[ 0.10521753,  0.1010392 ],
         [ 0.07642272,  0.07521524],
         [ 0.01565283, -0.06953315]]],


       [[[ 0.67918026,  0.65220912],
         [ 0.49330948,  0.48551516],
         [ 0.1010392 , -0.4488372 ]],

        [[ 0.50559292,  0.48551516],
         [ 0.36722766,  0.36142544],
         [ 0.07521524, -0.33412177]],

        [[-0.46739819, -0.4488372 ],
         [-0.33948566, -0.33412177],
         [-0.06953315,  0.30888073]]]])

In [146]:
np.tensordot(matriz1bis,matriz1bisT,1) # nótese la diferencia cuando
# cambiamos el índice '0' por el '1'

array([[1.09604261, 0.97687477],
       [0.97687477, 1.3225153 ]])

In [147]:
np.kron(matriz1bis,matriz2bis)

array([[ 0.05415147,  0.18347255,  1.49039708,  0.03933188,  0.13326175,
         1.08252118,  0.00805592,  0.02729455,  0.22172101],
       [ 0.03806575, -0.92470292,  0.20700276,  0.02764832, -0.67164013,
         0.15035246,  0.0056629 , -0.13756473,  0.03079506],
       [ 0.05200104,  0.17618661,  1.43121146,  0.03871043,  0.1311562 ,
         1.06541727, -0.03578607, -0.12124808, -0.98493095],
       [ 0.03655411, -0.88798176,  0.19878241,  0.02721148, -0.66102818,
         0.14797689, -0.0251558 ,  0.61109119, -0.13679806]])

In [148]:
?np.kron

In [149]:
?np.einsum

### Normas matriciales y vectoriales

En el siguiente link podrá consultar información relevante acerca de las posibilidades de calcular con Python tanto [normas matriciales](https://docs.scipy.org/doc/numpy-1.10.4/reference/generated/numpy.linalg.norm.html) así como el [número de condición](https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.cond.html) de una matriz cuadrada inversible.

In [150]:
np.linalg.norm(matriz1),np.linalg.norm(matriz1.T)

(2.1894615897226273, 2.1894615897226273)

In [151]:
np.linalg.norm(matriz1,np.inf),np.linalg.norm(matriz1,-np.inf)

(2.3835381864230465, 1.3119143556884039)

In [152]:
np.linalg.norm(matriz1,'fro'),np.linalg.norm(matriz1,1)

(2.1894615897226273, 2.4742060504645336)

In [153]:
np.linalg.norm(matriz1,1, axis = 0)

array([2.47420605, 1.71872271, 1.75140214])

In [154]:
np.linalg.norm(matriz1,1, axis = 1)

array([2.24887836, 2.38353819, 1.31191436])

In [155]:
matriz1[0], np.linalg.norm(matriz1[0], 1)

(array([-0.98177445,  0.96752798,  0.29957593]), 2.2488783583999084)

In [156]:
matriz1[0], np.linalg.norm(matriz1[1], 1)

(array([-0.98177445,  0.96752798,  0.29957593]), 2.3835381864230465)

In [157]:
np.linalg.norm(matriz1, 1, axis = (0,1))

2.4742060504645336

In [158]:
np.linalg.norm(matriz1, 1, axis = (1,0))

2.3835381864230465

In [159]:
np.linalg.cond(matriz1,np.inf),np.linalg.cond(matriz1inv,np.inf)
# comprobamos cómo el número de condición (en este caso con la norma
# del infinito) de una matriz coincide con el de su traspuesta

(52.078296525857134, 52.07829652585717)

In [160]:
np.linalg.norm(matriz1,np.inf),np.linalg.norm(matriz1inv,np.inf)
# he aquí las correspondientes normas de la matriz y de su inversa

(2.3835381864230465, 21.849155521192024)

In [161]:
np.linalg.norm(matriz1,np.inf)*np.linalg.norm(matriz1inv,np.inf)
# de manera que su producto da lugar a su número de condición 

52.078296525857134

In [162]:
np.linalg.cond(matriz1,2),np.linalg.cond(matriz1,1)
# He aquí los números de condición de la misma matriz
# asociados a otras normas habituales  

(39.710251176212005, 77.60855136887731)

In [163]:
np.linalg.cond(matriz2) # a mayor número de condición, peor 
# condicionado estará el problema de resolución de cualquier 
# sistema lineal asociado con dicha matriz

3.2545347922139913

In [164]:
np.linalg.cond(matriz2,np.inf),np.linalg.cond(matriz2,1)

(5.431228634178116, 5.526888725879853)

## Ejercicios

1.- Considere la siguiente matriz cuadrada

$$
A=\left(
\begin{array}{cccc}
 2 & 4 & 3 & 1 \\
 5 & 6 & 4 & 3 \\
 7 & 3 & 1 & 9 \\
 2 & 3 & 2 & 1 \\
\end{array}
\right).$$

Halle su traspuesta, su determinante y su rango. ¿Es invertible? En caso afirmativo, halle su inversa. Asimismo, determine las matrices $A^2$ y $A^3$. 

2.- Calcule las $p$-normas (con $p=1,2,\infty$) de la matriz anterior con las órdenes directas de Python indicadas. 

3.- Calcule también el número de condicionamiento de dicha matriz a la hora de resolver sistemas lineales. Para ello emplee la orden directa de Python indicada.

4.- Genere un notebook o fichero Markdown de RStudio que contenga e implemente, usando el lenguaje R, gran parte de órdenes equivalentes a las mostradas aquí con Python. 